<a href="https://colab.research.google.com/github/zillmerhanne/complex-prediction/blob/main/Harnessing_PDBe_KB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDBe API Training

## Introduction

This interactive Python notebook will guide you through programmatically accessing Protein Data Bank in Europe (PDBe) data using our REST and Graph APIs.

The REST API is a programmatic way to obtain information from the PDB and EMDB archives. It allows you to access and filter the vast amount of data stored about the protein structures stored in the archives, indexed by PDB entity. The Graph API provides a similar function, although it is not indexed like a relational database, enabling more abstract queries that would otherwise require post-API processing.

Many types of information about the structures stored are made available through structured data categories. For example, you can access information about:
* sample details
* experimental setup
* model quality
* bound compounds
* assembly formation
* cross-references
* publications
* and much more...

For more information, visit https://pdbe.org/api

**Interactive cells contain code and are represented with the ▶️ symbol. Run these cells in order to define functions, query APIs and process the fetched data.**

Much of the code in this notebook has been hidden for this workshop. You can view the source code by clicking on the "Show code" buttons. If you would like to learn more about programmatically accessing data from the PDBe and PDBe-KB, we have a [freely availabile training repository on GitHub](https://github.com/PDBeurope/pdbe-api-training).


---

## 1.&nbsp; Initialise environment

In [ ]:
#@title 1.1 ▶️ Install and import libraries

#@markdown Run this code block to install all the necessary packages needed for accessing and processing API data.
#@markdown <br> <br>
#@markdown The following packages will be imported:
#@markdown * [requests](https://requests.readthedocs.io/en/master/): Querying APIs
#@markdown * [pprint](https://docs.python.org/3/library/pprint.html): Improved readability of JSON output
#@markdown * [solrq](https://pypi.org/project/solrq/): Formatting of query arguments to Solr APIs
#@markdown * [sys](https://docs.python.org/3/library/sys.html): System-specific parameters and functions
#@markdown * [google](): Handling data stored on Google Drive via Google Colab notebooks

!pip install solrq # &> /dev/null

# API access
from solrq import Q
from pprint import pprint
import sys
import requests

# Data processing
import pandas as pd
from matplotlib import pyplot as plt

#Google Colab specific imports
import os
from google.colab import drive


In [ ]:
#@title 1.2 ▶️ Define global variables

#@markdown Run this code block to initialise the API stem URIs as global variables. This will help with code readability and reduce repitition.
#@markdown <br>
#@markdown <br>
#@markdown If you are interested in working with the PDBe APIs, click on the "Show code" link to reveal the variables. You can otherwise complete this notebook without learning these URIs.
BASE_URL = "https://www.ebi.ac.uk/pdbe/"  # the beginning of the URL for PDBe's API.

# For Solr API quries in section 2
SEARCH_URL = BASE_URL + "search/pdb/select?"  # the rest of the URL used for PDBe's search API.

# For Graph API queries in section 3
PDBEKB_UNIPROT_URL = BASE_URL + "graph-api/uniprot/"
PDBEKB_UNIPROT_LIGAND_URL = PDBEKB_UNIPROT_URL + "ligand_sites/"

# For Graph API queries in section 4
PDBEKB_UNIPROT_INTERACTION_URL = PDBEKB_UNIPROT_URL + "interface_residues/"

# For Graph API queries in section 5
PDBEKB_UNIPROT_COMPLEX_URL = PDBEKB_UNIPROT_URL + "complex/"

# For Graph API queries in section 6
PDBEKB_UNIPROT_SIMILARITY_URL = PDBEKB_UNIPROT_URL + "similar_proteins/"


In [ ]:
#@title 1.3 ▶️ Define useful functions

#@markdown You do not need to pour over the source code here, but know that running this block will instantiate several useful functions that we will use to retrieve data from the PDBe's Solr and Graph APIs.
#@markdown <br> <br>
#@markdown The functions you are initialising are as follows:
#@markdown <br>
#@markdown * **Section 2:**
#@markdown  * `format_search_terms_post()`: Parses Solr object into API-readable dictionary
#@markdown  * `make_request_post()`: Sends post request to PDBe API
#@markdown  * `run_search()`: Accepts query and filter terms, running the two functions above
#@markdown * **Section 3:**
#@markdown  * `get_url()`: Makes a request to a Graph API URL. Returns a JSON of the results
#@markdown  * `get_ligand_site_data()`: Retrieve ligand site data for a given UniProt accession and parse it into list.
#@markdown  * `explode_dataset()`: Expands a dataset with column elements stored as lists.
#@markdown * **Section 4:**
#@markdown  * `get_macromolecule_interaction_data()`: Get all the macromolecule interaction data for a given UniProt accession
#@markdown * **Section 5:**
#@markdown  * `get_complexes_protein_data()`: Get all the protein complexes observed in PDB entries for a given UniProt accession
#@markdown * **Section 6:**
#@markdown  * `get_similar_protein_data()`: Get similar protein data for a given UniProt accession and identity threshold



# --- Section 2 functions ---

def format_search_terms_post(search_terms, filter_terms=None, **kwargs):
    """
    Formats the search terms for the PDBe API
    """

    # Variable to return
    return_variables = {'q': str(search_terms)}

    if filter_terms:
        fl = ','.join(filter_terms)
        return_variables['fl'] = fl

    for arg in kwargs:
        return_variables[arg] = kwargs[arg]

    return return_variables

def make_request_post(search_dict, number_of_rows=10):
    """
    Makes a get request to the PDBe API
    """

    if 'rows' not in search_dict:
        search_dict['rows'] = number_of_rows

    search_dict['wt'] = 'json'
    response = requests.post(SEARCH_URL, data=search_dict)

    if response.status_code == 200:
        return response.json()

    else:
        print(f"[No data retrieved - {response.status_code}] {response.text}")

    return {}

def run_search(search_terms, filter_terms=None, number_of_rows=10, **kwargs):
    """
    Run the search with set of search terms
    """

    search_params = format_search_terms_post(search_terms=search_terms, filter_terms=filter_terms)

    if search_params:
        response = make_request_post(search_dict=search_params, number_of_rows=number_of_rows)

        if response:
            results = response.get('response', {}).get('docs', [])
            print(f'Number of results for {search_terms}: {len(results)}')
            return results

    print('No results')
    return []



# --- Section 3 functions ---

def get_url(url):
    """
    Makes a request to a URL. Returns a JSON of the results
    """
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()

    else:
        print(f"[No data retrieved - {response.status_code}] {response.text}")

    return {}

def get_ligand_site_data(uniprot_accession):
    """
    Retrieve ligand site data for a given UniProt accession and parse it into list.
    """

    url = PDBEKB_UNIPROT_LIGAND_URL + uniprot_accession

    data = get_url(url=url)
    data_to_ret = []

    for data_uniprot_accession in data:
        accession_data = data.get(data_uniprot_accession)

        for row in accession_data['data']:
            ligand_accession = row['accession']
            name = row['name']

            # Get the number of atoms in the ligand, return {} if not found
            num_atoms = row.get('additionalData', {})
            num_atoms = num_atoms['numAtoms']

            for residue in row["residues"]:
                residue['ligand_accession'] = ligand_accession
                residue['ligand_name'] = name
                residue['ligand_num_atoms'] = num_atoms
                residue['uniprot_accession'] = uniprot_accession

                # Calculate the interaction ratio, empty list if no interaction data
                residue['interaction_ratio'] = len(
                    residue.get('interactingPDBEntries', [])
                ) / len(
                    residue.get('allPDBEntries', [])
                )

                data_to_ret.append(residue)

    return data_to_ret

def explode_dataset(dataset, column_to_explode=None):
    """
    Expands a dataset (input as a dictionary) where a given column (column_to_explode)
    contains a list of values. Exploding this column will result in a new row for each
    elements in the list. Where onlu a single element is in the list, the row will not
    be duplicated.
    """

    df = pd.DataFrame(dataset)
    if column_to_explode:
        df = df.explode(column=column_to_explode).reset_index(drop=True)

    else:
        for column in df.select_dtypes(include='object'):
            df = df.explode(column=column).reset_index(drop=True)

    return df



# --- Section 4 functions ---

def get_macromolecule_interaction_data(uniprot_accession):
    """
    Get all the macromolecule interaction data for a given UniProt accession
    """

    url = PDBEKB_UNIPROT_INTERACTION_URL + uniprot_accession

    data = get_url(url=url)
    data_to_ret = []

    for data_uniprot_accession in data:
        accession_data = data.get(data_uniprot_accession)
        length = accession_data['length']

        for row in accession_data['data']:
            interaction_accession = row['accession']
            all_pdb_entries = row['allPDBEntries']
            name = row['name']

            # Get the accession type, return empty dictionary if not present
            accession_type = row.get('additionalData', {})
            accession_type = accession_type['type']

            for residue in row.get('residues', []):
                residue['interaction_accession'] = interaction_accession
                residue['interaction_name'] = name
                residue['length'] = length
                residue['uniprot_accession'] = uniprot_accession
                residue['interaction_accession_type'] = accession_type

                # Get the interacting PDB entries, return empty list if not present
                interacting_entries = residue.get('interactingPDBEntries', [])
                residue['interactingPDBEntries'] = interacting_entries
                residue['interaction_ratio'] = len(interacting_entries) / len(all_pdb_entries)
                del residue['allPDBEntries']
                data_to_ret.append(residue)

    return data_to_ret



# --- Section 5 functions ---

def get_complexes_protein_data(accession):
    """
    Get all the protein complexes observed in PDB entries for a given UniProt accession
    """

    url = PDBEKB_UNIPROT_COMPLEX_URL + accession

    dictfilt = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])
    data = get_url(url=url)
    data_out = []

    for row in data[accession]:

        # Get complex ID for row
        complex_id = list(row.keys())[0]

        my_row = {"complex_id": complex_id}
        my_row.update(row[complex_id])

        # Example of list comprehension to quickly create a list
        necc_rows = [keys for keys in my_row.keys() if keys !='participants']
        necc_rows = dictfilt(my_row,necc_rows)

        for item in my_row['participants'] :
            # Example of dictionary comprehension to quickly create a dictionary
            dict3 = {k:v for d in (necc_rows,item) for k,v in d.items()}
            data_out.append(dict3)

    return data_out



# --- Section 6 functions ---

def get_similar_protein_data(accession, identity):
    """
    Get similar protein data for a given UniProt accession and identity threshold
    """

    url = PDBEKB_UNIPROT_SIMILARITY_URL + accession + '/' + str(identity)

    data = get_url(url=url)

    return data


---

## 2.&nbsp; Retrieve PDB entries via Solr REST API

The Solr API is an excellent way to programmatically retrieve PDB entities from the archive, based on a set of search criteria. If your requirement for data follows "_I need `<a piece of/all information>` on all the PDB structures that `<satisfy a condition`>_", then the Solr API is your point of call.

In [ ]:
#@title 2.1 ▶️ Fetch entries using a single field -- *molecule name*

#@markdown Searching for all the PDB entries that have a particular "molecule name" can be easily achieved via the PDBe's Solr API. By default, only the top-ten hits are returned, but this can be increased without bound.
#@markdown <br> <br>
#@markdown **NB:** *The PDB currently contains over 215k entries. When writing your own queries, test with the default number as returning large amounts of data can be slow.*

# Create the Solr search terms object
input_molecule_name = 'Acetylcholinesterase' #@param {type:"string"}
search_terms = Q(molecule_name=input_molecule_name)

# Run the search
initial_results = run_search(search_terms)
print("Finished")


In [ ]:
#@markdown 🖨️ **Show results...**
pprint(initial_results[0])

In [ ]:
#@title 2.2 ▶️ Fetch entries using two fields -- *molecule name* and *organism name*

#@markdown Multiple fields can be queried at the same time to refine a search, retrieving more specific results. Again, only the top-ten hits are returned but we will later look at how this can be increased.
input_molecule_name = 'Acetylcholinesterase' #@param {type:"string"}
input_organism_name = 'Homo sapiens' #@param {type:"string"}

search_terms = Q(organism_name=input_organism_name, molecule_name=input_molecule_name)
multi_field_results = run_search(search_terms)
print("Finished")


In [ ]:
#@markdown 🖨️ **Show results...**
pprint(multi_field_results[0])

In [ ]:
#@title 2.3 ▶️ Fetch entries with two custom fields

#@markdown To give you an impression of the actionable queries, this code block offers some preselected fields to customise your query, refining the variety of results returned. For a complete list of available fields, refer to the PDBe's [Solr API documentation page](https://www.ebi.ac.uk/pdbe/api/doc/search.html).
#@markdown <br><br>
#@markdown **NB:** *Once you are satisfied toggling between the pre-selected search fields, you can add more query terms by adding additional arguments to the `Q(...)` object in the code editor.*
#@markdown <br><br>
#@markdown **First query field:**
input_field_name_1 = "molecule_name" #@param ["molecule_name", "organism_name", "uniprot_accession", "enzyme_name", "go_id", "resolution", "pubmed_id", "deposition_year"]
input_field_value_1 = "Acetylcholinesterase" #@param {type:"string"}

#@markdown **Second query field:**
input_field_name_2 = "organism_name" #@param ["molecule_name", "organism_name", "uniprot_accession", "enzyme_name", "go_id", "resolution", "pubmed_id", "deposition_year"]
input_field_value_2 = "Homo sapiens" #@param {type:"string"}

custom_field_results = run_search(search_terms)
print("Finished")

In [ ]:
#@markdown 🖨️ **Show results...**
pprint(custom_field_results[0])

In [ ]:
#@title 2.4 ▶️ Fetch and filter entries on API call

#@markdown So far, we have been retrieving all information pertaining to the valid PDB entries for a given set of query fields. This includes author names, publication abstract, entry DOI, crystallographic details (if a crystal structure), and more. Although valuable, we often need only a subset of this information. Relevant information can be "filtered" out from the returned JSON objects, or obtained at the time of calling the API. Run this code block to compare the results obtained here, vs those from section **2.1**.
# Create the Solr search terms object
input_molecule_name = 'Acetylcholinesterase' #@param {type:"string"}
search_terms = Q(molecule_name=input_molecule_name)

#@markdown **Filter terms:**
pdb_id = True #@param {type:"boolean"}
resolution = False #@param {type:"boolean"}
release_year = False #@param {type:"boolean"}

# Compile filter terms from booleans into list
filter_term_dict = {
    "pdb_id": pdb_id,
    "resolution": resolution,
    "release_year": release_year
}
filter_terms = [key for key, value in filter_term_dict.items() if value]

# Run the search
filtered_results = run_search(search_terms, filter_terms)
print("Finished")




In [ ]:
#@markdown 🖨️ **Show results...**
pprint(filtered_results)

The information returned from this API call should now contain only information requested by the filter terms, whilst still matching the conditions of the query terms (only `molecule_name` here).

In [ ]:
#@title 2.4 ▶️ Fetching more than 10 entries

#@markdown Our API calls until now have returned only the top-ten hits from the PDB archive, not necessarily in any order. By filtering the results at the point of calling the API, we can safely increase this default ten-entry limit as high as we like (note, there are over 215k entries in the PDB archive) without suffering an extensively large time penalty. Of course, complex queries with extensive filter terms will process slower, so it is advisable to test with the default number before increasing this limit.
#@markdown <br><br>
#@markdown We will continue to use `molecule_name` as our only query field and use three filter terms: `pdb_id`, `resolution` and `release_year`.
input_molecule_name = 'Acetylcholinesterase' #@param {type:"string"}
search_terms = Q(molecule_name=input_molecule_name)

filter_terms = ["pdb_id", "resolution", "release_year"]
max_entry_number = 1000 #@param {type:"integer"}

many_hit_results = run_search(search_terms, filter_terms, number_of_rows=max_entry_number)

#@markdown **NB:** *The actual number of entries returned (hits) might be less than the `max_entry_number`, depending on the number of structures that meet the query field(s).*

In [ ]:
#@markdown 🖨️ **Show results...**
print(len(many_hit_results))
pprint(many_hit_results)

---

## 3.&nbsp; Retrieve ligand information from PDBe-KB Graph API

In section 2, we retrived information on a per-entry basis. Here, we will access aggregated information across the whole PDB archive from several different biofunctional perspectives. Aggregated structural data can tell you not just about a single structure, but rather retrieve information pertaining to all entries across the entire archive, without extensive post-processing.

In turn, we will look at retrieving information on ligand binding, macromolecular interactions, complexes, related proteins and predicted models. If you need to make a query not currently available via these APIs, the PDBe-KB's complete graph database, which can be [downloaded in full here](https://www.ebi.ac.uk/pdbe/pdbe-kb/graph). Documentation on the [graph database's schema is made available on the PDBe-KB website](https://www.ebi.ac.uk/pdbe/pdbe-kb/schema). Detailed documentation on the [Cypher queries used by the APIs are also made available](https://wwwdev.ebi.ac.uk/pdbe/aggregated-api/docs/api.html).

In [ ]:
#@title 3.1 ▶️ Fetch ligand information for a given UniProt accession
#@markdown Query the PDBe-KB Graph API for all ligand in the PDB archive that interact with the given UniProt accession.

input_uniprot_accession = 'P22303' #@param {type:"string"}
ligand_data = get_ligand_site_data(uniprot_accession=input_uniprot_accession)

In [ ]:
#@markdown 🖨️ **Show results...**
pprint(ligand_data)

Unlike the Solr API, the results from the graph API are not limited to the top-ten hits. You will retrieve data from all hits at once, which may require parsing in a subsequent step. We will look at an example of this using the Pandas library below.

In [ ]:
#@title 3.2 ▶️ Parse ligand results from API into a Pandas DataFrame
#@markdown The results obtained from the graph API query are in JSON format. Run this cell to parse these data into a tabular Pandas DataFrame to make for easier data wrangling.

df_ligand_data = explode_dataset(
    dataset = ligand_data,
    # Uncomment below argument to explode a single column
    # column_to_explode='interactingPDBEntries'
)
df_ligand_data.head()

In [ ]:
#@markdown The `interactingPDBEntries` column contains its results in dictionary format. Let's separate the keys and values out into their own cells by running this cell.
lig_prot_interactions = pd.json_normalize(df_ligand_data['interactingPDBEntries'])
df_lig_prot_interactions = df_ligand_data.join(lig_prot_interactions)
df_lig_prot_interactions = df_lig_prot_interactions.drop(columns='interactingPDBEntries')
df_lig_prot_interactions.head()

Now you have your ligand association results in tabular form, they can be saved to file (e.g. `df.to_csv("file_name.csv")`) and/or data mined. We will next look look at how these data can be visualised.

In [ ]:
#@title 3.3 ▶️ Visualising parsed ligand association results

#@markdown Run this cell to generate a scatter plot of ligand interaction frequency over the UniProt residue range.
res_interactions = df_lig_prot_interactions.groupby('startIndex')['pdbId']
res_interactions_count = res_interactions.count().reset_index()
res_interactions_count.plot.scatter(x='startIndex', y='pdbId', xlabel="UniProt residue ID", ylabel='# ligand interactions')


In [ ]:
#@markdown Calculate some measures of central tendancy and spread to identify residues with high interaction frequency. Hopefully, these examples give you an impression of the data retrievable by this API.

mean = res_interactions_count.mean()
std = res_interactions_count.std()

mean_value = float(mean.values[1])
std_value = float(std.values[1])
two_std_value = std_value * 2


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5), tight_layout=True) # this makes one plot with an axis "ax" which we can add several plots to

lt_data = res_interactions_count[res_interactions_count['pdbId'] <= mean_value]
lt_data.plot.scatter(
    x='startIndex',
    y='pdbId',
    color='blue',
    ax=ax1
)

gt_data = res_interactions_count[res_interactions_count['pdbId'] > mean_value]
gt_data.plot.scatter(
    x='startIndex',
    y='pdbId',
    color='red',
    ax=ax1
)
ax1.axhline(mean_value, label='Mean')
ax1.text(80, mean_value+50, "Mean", ha='right', va='bottom')


lt_data = res_interactions_count[res_interactions_count['pdbId'] <= two_std_value]
lt_data.plot.scatter(
    x='startIndex',
    y='pdbId',
    color='blue',
    ax=ax2
)

gt_data = res_interactions_count[res_interactions_count['pdbId'] > two_std_value]
gt_data.plot.scatter(
    x='startIndex',
    y='pdbId',
    color='red',
    ax=ax2
)
ax2.axhline(two_std_value)
ax2.text(80, two_std_value+50, "2\u03C3", ha='right', va='bottom')

for ax in (ax1, ax2):
  ax.set_ylabel('# ligand interactions')
  ax.set_xlabel("UniProt residue ID")
  ax.grid(alpha=0.3)

plt.show()
plt.close()



## 4.&nbsp; Retrieve macromolecular interaction information from PDBe-KB Graph API

We will next look at collecting information on residue-residue interactions between macromolecules within assemblies, across the PDB archive.



In [ ]:
#@title 4.1 ▶️ Fetch macromolecule interaction information for a given UniProt accession
#@markdown Query the PDBe-KB Graph API for all macromolecules in the PDB archive that interact with the structures of the given UniProt accession.

input_uniprot_accession = 'P22303' #@param {type:"string"}
interaction_data = get_macromolecule_interaction_data(uniprot_accession=input_uniprot_accession)



In [ ]:
#@markdown 🖨️ **Show results...**
pprint(interaction_data)

In [ ]:
#@title 4.2 ▶️ Parse interaction results from the API into a Pandas DataFrame
#@markdown Similarly to the ligand data in section 3, the results obtained from the graph API query are in JSON format. Run this cell to parse these data into a tabular Pandas DataFrame to make for easier data wrangling.

df_all_interactions = explode_dataset(
    dataset=interaction_data,
    # Uncomment to explode a single column
    column_to_explode='interactingPDBEntries'
)
df_all_interactions.head()


In [ ]:
#@markdown Like with the ligand interaction data, we also have some columns that contain dictionaries, that we would like to extract to fill new rows.

data = pd.json_normalize(df_all_interactions['interactingPDBEntries'])

# Add the new columns to the dataframe
df_interactions = df_all_interactions.join(data)
# Remove some columns that are not needed
df_interactions = df_interactions.drop(columns='interactingPDBEntries')
df_interactions.head()

In [ ]:
#@markdown Plot frequency of macromolecular interaction counts across the given UniProt residue range.
res_macromol_interactions = df_interactions.groupby('startIndex')['pdbId']
res_macromol_interactions_count = res_macromol_interactions.count().reset_index()
res_macromol_interactions_count.plot.scatter(
  x='startIndex', y='pdbId', xlabel="UniProt residue ID", ylabel='# ligand interactions'
)
#@markdown It's now visually clear there are two regions in this protein that interact frequently with other macromolecules, at least as found in the PDB archive.

In [ ]:
#@markdown Plot frequency of macromolecular interaction counts by interaction partner.
fig, ax = plt.subplots(figsize=(5, 5), tight_layout=True)

for interacting_unp in df_interactions["interaction_accession"].unique():
  data = df_interactions[df_interactions['interaction_accession'] == interacting_unp]
  data = data.groupby('startIndex')['pdbId']
  data_count = data.count().reset_index()

  ax.scatter(data_count["startIndex"], data_count["pdbId"], label=interacting_unp)

ax.set_xlabel("UniProt residue ID")
ax.set_ylabel("# macromolecule interactions")

ax.legend()
ax.grid(alpha=0.3)
plt.show()
plt.close()

#@markdown What does this view tell you about the interaction of this protein with other macromolecules? You can search for the UniProt accessions via the [PDBe-KB website](https://www.ebi.ac.uk/pdbe/pdbe-kb/) to learn more about the interacting partners.



## 5.&nbsp; Retrieve complexes from PDB archive via the Graph API



In [ ]:
#@title 5.1 ▶️ Fetch complexes information for a given UniProt accession
#@markdown Query the PDBe-KB Graph API for all macromolecules in the PDB archive that interact with the structures of the given UniProt accession.

input_uniprot_accession = 'P43681' #@param {type:"string"}
complexes_results = get_complexes_protein_data(input_uniprot_accession)

In [ ]:
#@markdown 🖨️ **Show results...**
pprint(complexes_results)

In [ ]:
#@markdown Run this cell to transform the API results into a Pandas DataFrame. A graph of the total number of chains found in each complex your UniProt accession is found in.

# Transform into table
df_complexes_with_duplicates = pd.DataFrame(complexes_results)

# Sum the stoicheometry of each complex ID and plot
df_complexes_grouped = df_complexes_with_duplicates.groupby('complex_id')
df_complexes_stoich_sum = df_complexes_grouped["stoichiometry"].sum().sort_values()
df_complexes_stoich_sum.plot.bar(
  x='complex_id', y='accession', ylabel=f"# occurences of {input_uniprot_accession} in complexes"
)

## 6.&nbsp; Retrieve similar proteins from PDB archive via the Graph API

The API call you will look at here is able to retrieve all structures from the PDB archive that meet a given threshold of sequence identity. This is similar to the search you performed earlier using the FoldSeek feature on the AlphaFold Database. Although, the results returned here are for experimentally derived structures only and are found via sequence identity, rather than structural similiarity.



In [ ]:
input_uniprot_accession = 'P22303' #@param {type:"string"}
identity_cutoff = 50 # @param {type:"slider", min:1, max:100, step:1}
similarity_results = get_similar_protein_data(input_uniprot_accession, identity_cutoff)

In [ ]:
#@markdown 🖨️ **Show results...**
pprint(similarity_results)

In [ ]:
#@markdown Run this cell to summarise how many hits you obtained at 50 % sequence identity. The output shows the number of hits by species name, molecule name and sequence identity (the API returns all structures with seq_id>=50).
#@markdown <br><br>
#@markdown You should only obtain 6 chains (as of September 2024). Reduce the sequence identity and the run the API call again to obtain more hits.

# Reformat data to make it a list of similar proteins for the protein of interest
df_expanded_uniprot = explode_dataset(
    dataset=similarity_results[input_uniprot_accession],
    column_to_explode='mapped_segment'
)

# Obtain the flattened data for each similar protein
data = pd.json_normalize(df_expanded_uniprot['mapped_segment'])

# Create reformatted dataset by joining the flattened data with the exploded dataset
df_similar_proteins = df_expanded_uniprot.join(data)
df_similar_proteins = df_similar_proteins.drop(columns='mapped_segment')

for col in ("species", "description", "sequence_identity"):
  print(df_similar_proteins[col].value_counts(), '\n')

df_similar_proteins